In [6]:
# Adopted from 
# https://github.com/watson-developer-cloud/python-sdk/blob/master/examples/visual_recognition_v4.py

import json
import os
from ibm_watson import VisualRecognitionV4
from ibm_watson.visual_recognition_v4 import FileWithMetadata, TrainingDataObject, Location, AnalyzeEnums
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


In [9]:
authenticator = IAMAuthenticator('UNj_AQojzpzeg5q5FvmaMu2a3jathuqDB79_DjpAGJb_')

service = VisualRecognitionV4(
    '2020-03-20',
    authenticator=authenticator)
service.set_service_url('https://api.us-south.visual-recognition.watson.cloud.ibm.com/instances/de0d73f8-faaa-4318-9ed6-492a2822bf20')

# create a classifier
my_collection = service.create_collection(
    name='obj_detection_test',
    description='testing for python obj detection'
).get_result()
collection_id = my_collection.get('collection_id')


In [17]:
# Modify image path
# os.path.join(os.path.abspath(''), 'South_Africa_Luca_Galuzzi_2004.jpeg')


'/Users/dqin/Documents/FAME/watson_experiment/South_Africa_Luca_Galuzzi_2004.jpeg'

In [18]:
# add images (i.e. upload image to IBM)
with open(os.path.join(os.path.abspath(''), 'resources/South_Africa_Luca_Galuzzi_2004.jpeg'), 'rb') as giraffe_info:
    add_images_result = service.add_images(
        collection_id,
        images_file=[FileWithMetadata(giraffe_info)],
    ).get_result()
print(json.dumps(add_images_result, indent=2))
image_id = add_images_result.get('images')[0].get('image_id')

{
  "images": [
    {
      "image_id": "South_Africa_Luca_Galuzzi_2004_202349062c2307571a3f7edc71fe819f",
      "created": "2020-03-20T15:12:12.768Z",
      "updated": "2020-03-20T15:12:12.768Z",
      "source": {
        "type": "file",
        "filename": "South_Africa_Luca_Galuzzi_2004.jpeg"
      },
      "dimensions": {
        "height": 853,
        "width": 1280
      },
      "training_data": {}
    }
  ]
}


In [19]:
# add image training data (label out the )
training_data = service.add_image_training_data(
    collection_id,
    image_id,
    objects=[
        TrainingDataObject(object='giraffe training data',
                           location=Location(64, 270, 755, 784))
    ]).get_result()
print(json.dumps(training_data, indent=2))


{
  "objects": [
    {
      "object": "giraffe training data",
      "location": {
        "left": 270,
        "top": 64,
        "width": 755,
        "height": 784
      }
    }
  ]
}


In [20]:
# update object metadata
updated_object_metadata = service.update_object_metadata(
    collection_id=collection_id,
    object='giraffe training data',
    new_object='updated giraffe training data').get_result()
print(json.dumps(updated_object_metadata, indent=2))


{
  "object": "updated giraffe training data",
  "count": 1
}


In [21]:
# train collection
train_result = service.train(collection_id).get_result()
print(json.dumps(train_result, indent=2))


{
  "collection_id": "47e70760-dff3-4ebe-9098-3298b8f5b918",
  "name": "obj_detection_test",
  "description": "testing for python obj detection",
  "created": "2020-03-20T15:04:52.299Z",
  "updated": "2020-03-20T15:16:22.357Z",
  "image_count": 1,
  "training_status": {
    "objects": {
      "ready": false,
      "in_progress": true,
      "data_changed": false,
      "latest_failed": false,
      "description": "Training job queued."
    }
  }
}


In [27]:
# training usage
training_usage = service.get_training_usage()
print(json.dumps(training_usage.get_result(), indent=2))
# training_usage.get_headers()

{
  "start_time": "2019-10-01T00:00:00.000Z",
  "end_time": "2020-03-20T23:59:59.000Z",
  "completed_events": 0,
  "trained_images": 0,
  "events": []
}


In [29]:
# analyze
dog_path = os.path.join(os.path.join(os.path.abspath(''), 'resources/dog.jpg'))
giraffe_path = os.path.join(os.path.join(os.path.abspath(''), 'resources/my-giraffe.jpeg'))
with open(dog_path, 'rb') as dog_file, open(giraffe_path, 'rb') as giraffe_files: # open two files at once...
    analyze_images = service.analyze(
        collection_ids=[collection_id],
        features=[AnalyzeEnums.Features.OBJECTS.value],
        images_file=[
            FileWithMetadata(dog_file),
            FileWithMetadata(giraffe_files)
        ],
        image_url=['https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/American_Eskimo_Dog.jpg/1280px-American_Eskimo_Dog.jpg']).get_result()
    print(json.dumps(analyze_images, indent=2))


{
  "images": [
    {
      "source": {
        "type": "file",
        "filename": "dog.jpg"
      },
      "dimensions": {
        "height": 1174,
        "width": 2048
      },
      "objects": {
        "collections": [
          {
            "collection_id": "47e70760-dff3-4ebe-9098-3298b8f5b918",
            "objects": [
              {
                "object": "updated giraffe training data",
                "location": {
                  "left": 349,
                  "top": 18,
                  "width": 1417,
                  "height": 1108
                },
                "score": 0.978303
              }
            ]
          }
        ]
      }
    },
    {
      "source": {
        "type": "file",
        "filename": "my-giraffe.jpeg"
      },
      "dimensions": {
        "height": 615,
        "width": 410
      },
      "objects": {
        "collections": [
          {
            "collection_id": "47e70760-dff3-4ebe-9098-3298b8f5b918",
            "objects": [

In [ ]:


# delete collection
# service.delete_collection(collection_id)